In [15]:
import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Regression Classifier.
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_squared_error

from itertools import combinations
import pandas as pd

In [4]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

# TODO:  Correct features and assign target.
data = pd.read_csv(path + "/" + file)
data.columns
X = data[['Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing']]

y = data['Life_expectancy']

# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Path to dataset files: /Users/nathanlonghurst/.cache/kagglehub/datasets/lashagoch/life-expectancy-who-updated/versions/1


In [13]:
independent_columns = ['Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing']

best_bic = np.inf
best_aic = np.inf

for num_features in range(3, len(independent_columns) + 1):
    for combo in combinations(independent_columns, r=num_features):
        temp_X = X_train[list(combo)]
        X = sm.add_constant(temp_X)
        model = sm.OLS(y_train, X).fit()
        if model.aic < best_aic:
            best_aic = model.aic
            best_aic_features = list(combo)
        if model.bic < best_bic:
            best_bic = model.bic
            best_bic_features = list(combo)


model_aic = sm.OLS(y_train, X_train[best_aic_features]).fit()
model_bic = sm.OLS(y_train, X_train[best_bic_features]).fit()

ValueError: shapes (573,17) and (11,) not aligned: 17 (dim 1) != 11 (dim 0)

In [22]:
y_hat = model_aic.predict(X_test[best_aic_features])
print(f"mean squared error for best aic model: {mean_squared_error(y_test, y_hat)}")

y_hat = model_bic.predict(X_test[best_bic_features])
print(f"mean squared error for best bic model: {mean_squared_error(y_test, y_hat)}")

mean squared error for best aic model: 18.894208517795533
mean squared error for best bic model: 21.003520068961222


In [27]:
print(f"Best aic model had {len(best_aic_features)} features")
print(f"Best bic model had {len(best_bic_features)} features")

print(f"Best aic features: {best_aic_features}")
print(f"Best bic features: {best_bic_features}")

Best bic model had 10 features
Best aic model had 11 features
Best bic features: ['Infant_deaths', 'Under_five_deaths', 'Adult_mortality', 'Alcohol_consumption', 'BMI', 'Incidents_HIV', 'GDP_per_capita', 'Thinness_ten_nineteen_years', 'Schooling', 'Economy_status_Developing']
Best aic features: ['Infant_deaths', 'Under_five_deaths', 'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'BMI', 'Incidents_HIV', 'GDP_per_capita', 'Thinness_ten_nineteen_years', 'Schooling', 'Economy_status_Developing']


In [ ]:
model_aic.save("LinearRegression.pickle")